In [61]:
import pandas as pd
import os 
allncs = "All NCs (New) Non-conformance data YTD.xlsx"
allncs = pd.read_excel(allncs)
allncs.head()

,SupplierName,NonconformanceNumber,Disposition,FiscalPeriod,DispositionQty,PartNumber,PartName,NonconformanceDescription,PONumber,Pattern,UOM,NONCONFORMANCE_SOURCE,SiteNo,SiteName
0,NaN,NC000508022,SCRAP,P09,1,1U388835162,"STEM,PLUG ~ VSC1L3750,3/8X 8.88,2.38THD",Drill bit stuck in plug/stem,NaN,NaN,EA,INPROCESS,123,Sherman
1,NaN,NC000508205,SCRAP,P09,1,1U388835162,"STEM,PLUG ~ VSC1L3750,3/8X 8.88,2.38THD",stem was pulled wrong on job and already drilled,NaN,NaN,EA,INPROCESS,123,Sherman
2,NaN,NC000507834,SCRAP,P09,1,1U388835162,"STEM,PLUG ~ VSC1L3750,3/8X 8.88,2.38THD",Broke carbide drill bit in stem and plug,NaN,NaN,EA,INPROCESS,123,Sherman
3,NaN,NC000507094,SCRAP,P09,1,1U388835162,"STEM,PLUG ~ VSC1L3750,3/8X 8.88,2.38THD",Threads damaged during assembly.,NaN,NaN,EA,INPROCESS,123,Sherman
4,NaN,NC000507350,SCRAP,P09,1,1U388835162,"STEM,PLUG ~ VSC1L3750,3/8X 8.88,2.38THD",drill bit broke off insde,NaN,NaN,EA,INPROCESS,123,Sherman


In [57]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm
import requests
import json
tqdm.pandas()
def preprocess_dataframe(df, num_rows, columns_embed=[], model_name="all-MiniLM-L6-v2"):
    """
    Preprocess a DataFrame to add a 'vector' column with vector representations of concatenated column values.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        num_rows (int): The number of rows to process.
        columns_embed (list): List of column names to embed.
        model_name (str): The pre-trained embedding model to use.
        
    Returns:
        pd.DataFrame: A new DataFrame with an additional 'vector' column.
    """
    # Load the embedding model
    model = SentenceTransformer(model_name)
    if num_rows == -1 :
       num_rows = len(df) 
    else:
    # Ensure the specified number of rows is within bounds
        num_rows = min(num_rows, len(df))
    
    # Slice the DataFrame to process only the specified number of rows
    df = df.iloc[:num_rows].copy()
    
    # Concatenate specified columns into a single text string
    def concatenate_columns(row):
        return " ".join(str(row[col]) for col in columns_embed if pd.notnull(row[col]))
   
    # Apply concatenation and generate embeddings
    df["vector"] = df.apply(lambda row: model.encode(concatenate_columns(row)), axis=1)
    
    return df

def RA(messages, model):
    r = requests.post(
        "http://127.0.0.1:11434/api/chat",
        json={"model": model, "messages": messages, "stream": True},
        stream=True
    )
    r.raise_for_status()
    output = ""

    for line in r.iter_lines():
        body = json.loads(line)
        if "error" in body:
            raise Exception(body["error"])
        if body.get("done") is False:
            message = body.get("message", "")
            content = message.get("content", "")
            output += content
        if body.get("done", False):
            message["content"] = output
            return message

## K-Means

In [13]:
columns_to_embed = [
    "PartName", 
    "NonconformanceDescription", 
    "SiteName"
]  # Specify the columns to concatenate
num_rows = 10  # Process all rows in the example

processed_df = preprocess_dataframe(allncs, num_rows, columns_embed=columns_to_embed)
from sklearn.cluster import KMeans
import numpy as np
TOKENIZERS_PARALLELISM = True
# Prepare the vectors for clustering
vectors = np.vstack(processed_df["vector"].values)

# Perform K-Means clustering
num_clusters = 2  # Define the number of topics you expect
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
processed_df["cluster"] = kmeans.fit_predict(vectors)

# Print the cluster assignments
print(processed_df[["NonconformanceDescription", "cluster"]])
# Assign cluster labels (manually or with additional analysis)
topic_labels = {1: "Drill Bit Issue", 0: "Stem Issue"}
processed_df["topic"] = processed_df["cluster"].map(topic_labels)

print(processed_df[["NonconformanceDescription", "topic"]])

                          NonconformanceDescription  cluster
0                      Drill bit stuck in plug/stem        1
1  stem was pulled wrong on job and already drilled        1
2          Broke carbide drill bit in stem and plug        1
3                  Threads damaged during assembly.        0
4                         drill bit broke off insde        1
5                         Damaged durring assembly.        0
6                             Attached to bad plug.        0
7                              broken drill in plug        1
8                           Drill bit stuck inside.        1
9         Drill bit broke on the stem cannot remove        1
                          NonconformanceDescription            topic
0                      Drill bit stuck in plug/stem  Drill Bit Issue
1  stem was pulled wrong on job and already drilled  Drill Bit Issue
2          Broke carbide drill bit in stem and plug  Drill Bit Issue
3                  Threads damaged during assembly.  

In [67]:
columns_to_embed = [
    "PartName", 
    "PartNumber",
    "NonconformanceDescription", 
    "SiteName"
]  # Specify the columns to concatenate
num_rows = -1  # Process all rows in the example
tqdm.pandas()
processed_df = preprocess_dataframe(allncs, num_rows, columns_embed=columns_to_embed)

In [69]:
len(processed_df)

26017

## lanceDB

In [102]:
import lancedb
import pandas as pd
import pyarrow as pa
uri = "data/ncs-lancedb"
db = lancedb.connect(uri)
# db.drop_table("vectorizedncs")
ncs_tbl = db.create_table("vectorizedncs", processed_df)
ncs_tbl = db.open_table("vectorizedncs")
ncs_tbl


LanceTable(connection=LanceDBConnection(/Users/admin/source/quality-nir-classification/quality-nir-classification-api/data/ncs-lancedb), name="vectorizedncs")

In [ ]:
# Query text
model = SentenceTransformer("all-MiniLM-L6-v2")
query_text = "Leak test"
query_vector = model.encode(query_text)

# Perform the search
results = ncs_tbl.search(query_vector, vector_column_name="vector").limit(100).to_pandas()

# Display the results
columns_to_show = ["NonconformanceNumber","SupplierName","PartNumber", "PartName","NonconformanceDescription","SiteName"]
context = results[columns_to_show]
# Prepare the context for LLM query 

rows = []
for index, row in tqdm(context.iterrows(), total=len(context)):
        # Convert the row to a dictionary where the column names are keys
        row_dict = {col: str(row[col]) for col in context.columns}
        rows.append(row_dict)
rows_context = "\n".join(
            [f"{idx+1}. {json.dumps(row)}" for idx, row in enumerate(rows)]
        )

100%|██████████| 100/100 [00:00<00:00, 43383.37it/s]


In [ ]:
context


100%|██████████| 10/10 [00:00<00:00, 5393.22it/s]


In [ ]:

question = "Which suppliers are frequently reported in non conformaces"
rows_context = "\n".join(
            [f"{idx+1}. {json.dumps(row)}" for idx, row in enumerate(rows)]
        )
messages = [
            {
                "role": "system",
                "content": (
                    "You are an intelligent assistant specializing in analyzing non-conformance data. "
                    "Use the data provided as context to answer the question accurately."
                )
            },
            {
                "role": "user",
                "content": (
                    f"### Non-Conformance Data:\n{rows_context}\n\n"
                    f"### User Question:\n{question}\n\n"
                    "Provide a precise answer based on the provided non-conformaces, do not explain the analysis or write code."
                )
            }
        ]

        # Call the RA function
response = RA(messages=messages, model="wizardlm2")
response["content"]
# Return the insights

'Based on the provided non-conformances data, the suppliers that are frequently reported in non-conformances are as follows:\n\n1. **DIA-COM CORP** (for a single instance at the Singapore site)\n2. **None** (supplier not specified but multiple instances across different sites including Sherman, Singapore, and Mansfield)\n\nThe rest of the non-conformance entries do not specify a supplier, only indicating "SupplierName: None". However, it\'s important to note that in several cases, the part numbers are provided alongside the non-conformances which could potentially help identify the manufacturers if further investigation is conducted using those part numbers.'

In [100]:
print(response["content"])

Based on the provided non-conformances data, the suppliers that are frequently reported in non-conformances are as follows:

1. **DIA-COM CORP** (for a single instance at the Singapore site)
2. **None** (supplier not specified but multiple instances across different sites including Sherman, Singapore, and Mansfield)

The rest of the non-conformance entries do not specify a supplier, only indicating "SupplierName: None". However, it's important to note that in several cases, the part numbers are provided alongside the non-conformances which could potentially help identify the manufacturers if further investigation is conducted using those part numbers.


In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Concatenate relevant columns and generate embeddings
columns_to_embed = ["PartName", "NonconformanceDescription"]
df["vector"] = df.apply(
    lambda row: model.encode(" ".join(str(row[col]) for col in columns_to_embed if pd.notnull(row[col]))),
    axis=1,
)

/Users/admin/source/quality-nir-classification/quality-nir-classification-api/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/admin/source/quality-nir-classification/quality-nir-classification-api/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
# Example user input
user_input = "drill bit, stem issue, wrong assembly"

# Parse the topics
user_topics = [topic.strip() for topic in user_input.split(",")]

# Generate embeddings for each topic
topic_vectors = {topic: model.encode(topic) for topic in user_topics}